LAB 2

Imports

In [1]:
import numpy as np
from dataclasses import dataclass, field
import heapq
import random

Parameters:

In [2]:
#problem to resolve
file_name = "data/problem_g_100.npy"
array = np.load(file_name)

# number of cities to explore
INDIVIDUAL_VECTOR_SIZE =  array.shape[1]

#data class of the solution, we have the value of the solution and the vector to represent it
# vector: ordered list of cities to visit, vector[i] = 5 = array[5] is the i-th city to visit
@dataclass
class Individual:
    value: float
    vector: list[int]= field(default_factory=lambda: []*INDIVIDUAL_VECTOR_SIZE)
    
    def __lt__(self, other):
        return self.value < other.value

    def __repr__(self):
        return f"Individual({self.vector}, value={self.value})"
    
max_populations_size = 200
generations = 50000
first_population_size = 150
parent_selection_percentage = 0.3 #=30%
mutation_probability = 0.6 #=60%

if "10" in file_name or "20" in file_name or "50" in file_name:
    generations = 1000
if "100" in file_name:
    generations = 10000
if "200" in file_name:
    generations = 40000
if "500" in file_name or "1000" in file_name:
    generations = 50000

Functions:

In [3]:
##THE POPULATIONS IS A HEAP, SO WE ALWAYS INSERT NEGATIVE VALUE TO MAKE THE REMOVE FASTER

#Function to calculate the fitness of an individual
def calculate_fitness(individual: Individual):
    fitness = 0.0
    for i in range(len(individual.vector) - 1):
        city_from = individual.vector[i]
        city_to = individual.vector[i + 1]
        fitness += array[city_from][city_to]
    # Adding the return to the starting city
    fitness += array[individual.vector[-1]][individual.vector[0]]
    individual.value = -fitness

#Inizialization of a random population
def initialize_population(size = first_population_size) -> list[Individual]:
    population = []
    for _ in range(size):
        vector = np.random.permutation(INDIVIDUAL_VECTOR_SIZE).tolist()
        individual = Individual(value=0.0, vector=vector)
        calculate_fitness(individual)
        heapq.heappush(population, individual)
    return population

def parent_selection(population: list[Individual]) -> list[tuple[Individual, Individual]]:
    parents = heapq.nlargest(int(len(population) * parent_selection_percentage), population)
    random.shuffle(parents)
    pairs = [(parents[i], parents[i+1]) for i in range(0, len(parents)-1, 2)]
    return pairs
    
def survivor_selection(population: list[Individual]):
    while len(population) > max_populations_size:
        heapq.heappop(population)
        
def crossover(population: list[Individual], parent1: Individual, parent2: Individual) -> Individual:
    random_index = random.randint(0, INDIVIDUAL_VECTOR_SIZE - 1)
    if random_index == INDIVIDUAL_VECTOR_SIZE - 1:
        pair = (parent2.vector[random_index], parent2.vector[0])
    else:
        pair = (parent2.vector[random_index], parent2.vector[random_index + 1])
   
    child_vector = []
    for city in parent1.vector:
        if city not in pair:
            child_vector.append(city)
        else:
            if city == pair[0]:
                child_vector.append(pair[0])
                child_vector.append(pair[1])
    
    if random.random() < mutation_probability:
        idx1, idx2 = random.sample(range(INDIVIDUAL_VECTOR_SIZE), 2)
        child_vector[idx1], child_vector[idx2] = child_vector[idx2], child_vector[idx1]
        
    child = Individual(value=0.0, vector=child_vector)
    return child        

    
def recombination(population: list[Individual]):
    parents = parent_selection(population)
    for parent1, parent2 in parents:
        child = crossover(population, parent1, parent2)
        calculate_fitness(child)
        heapq.heappush(population, child)
    survivor_selection(population)
    

Resolving code:

In [4]:
population = initialize_population()
count = 0

for generation in range(generations):
    recombination(population)
    best_individual = heapq.nlargest(1, population)[0]
    if (generation) % (generations/10) == 0:
        print(f"Generation {generation}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")
    if (generation) % (generations/20) == 0:
        mutation_probability *= 0.95
    count += 1

print(f"Generation {generation+1}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")

Generation 0: Best Value = 22630.019540935373, Route = [22, 40, 43, 86, 74, 70, 45, 25, 8, 85, 35, 16, 11, 39, 82, 26, 84, 42, 46, 93, 55, 29, 68, 14, 34, 15, 88, 75, 64, 17, 66, 67, 19, 59, 49, 58, 65, 56, 98, 24, 44, 3, 37, 72, 0, 47, 1, 2, 31, 94, 52, 30, 89, 28, 62, 18, 90, 60, 12, 7, 96, 81, 54, 33, 99, 71, 63, 50, 76, 83, 57, 21, 27, 78, 10, 5, 92, 20, 51, 87, 97, 53, 95, 38, 73, 79, 6, 36, 80, 32, 4, 41, 13, 77, 48, 69, 9, 91, 61, 23]
Generation 1000: Best Value = 8995.352637579503, Route = [39, 19, 40, 6, 87, 86, 26, 35, 44, 73, 11, 58, 94, 82, 59, 12, 55, 93, 46, 36, 5, 37, 0, 14, 67, 15, 91, 34, 75, 7, 54, 32, 77, 25, 98, 65, 17, 79, 3, 51, 84, 2, 57, 42, 97, 30, 81, 18, 70, 66, 45, 53, 68, 89, 22, 64, 10, 33, 41, 74, 27, 20, 71, 47, 63, 99, 83, 96, 76, 29, 16, 31, 56, 28, 21, 60, 72, 48, 49, 69, 52, 62, 95, 38, 90, 13, 80, 24, 50, 85, 1, 8, 4, 43, 92, 88, 78, 61, 23, 9]
Generation 2000: Best Value = 7260.835912856639, Route = [35, 95, 61, 76, 13, 12, 75, 55, 46, 36, 49, 5, 3